## Transformation Data

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
data = pd.read_csv("dataset_1st/training.csv")

In [3]:
base_date = pd.to_datetime("2023-10-02")
data["locdt"] = base_date + pd.to_timedelta(data["locdt"], unit = "D")

In [4]:
data["loctm"] = data['loctm'].astype(str).str.zfill(6)
data['loctm'] = data['loctm'].str[:2] + ":" + data['loctm'].str[2:4] + ":" + data['loctm'].str[4:]
data["loctm"] = pd.to_datetime(data["loctm"], format = "%H:%M:%S")
data["loctm"] = data["loctm"].dt.time

In [5]:
data["lochr"] = data["loctm"].apply(lambda x: x.hour)

In [6]:
data["combined_datetime"] = pd.to_datetime(data["locdt"].astype(str)+" "+data["loctm"].astype(str))

In [7]:
contp_dum = pd.get_dummies(data["contp"], prefix = "contp", dtype = int)
data = pd.concat([data, contp_dum], axis = 1)

In [8]:
etymd_dum = pd.get_dummies(data["etymd"], prefix = "etymd", dtype = int)
data = pd.concat([data, etymd_dum], axis = 1)

In [9]:
def get_mcc_category(mcc_code):
    mcc_code = int(mcc_code)
    
    if 1 <= mcc_code <= 99:
        return "mcc1"
    elif 100 <= mcc_code <= 199:
        return "mcc2"
    elif 200 <= mcc_code <= 299:
        return "mcc3"  # You can continue this pattern for other ranges if needed
    elif 300 <= mcc_code <= 399:
        return "mcc4"
    elif 400 <= mcc_code <= 499:
        return "mcc5"
    else:
        return "unknown"

In [10]:
data["mcc_cat"] = data["mcc"].fillna(0).apply(get_mcc_category)
mcc_cat_dum = pd.get_dummies(data["mcc_cat"], prefix = "mcc_cat", dtype = int)
data = pd.concat([data, mcc_cat_dum], axis = 1)

In [11]:
data["conam_log"] = np.log(data["conam"] + 1)

In [12]:
data["flam1_log"] = np.log(data["flam1"] + 1)

In [13]:
stscd_dum = pd.get_dummies(data["stscd"], prefix = "stscd", dtype = int, dummy_na = True)
data = pd.concat([data, stscd_dum], axis = 1)

In [14]:
hcefg_dum = pd.get_dummies(data["hcefg"], prefix = "hcefg", dtype = int, dummy_na = True)
data = pd.concat([data, hcefg_dum], axis = 1)

In [15]:
data["csmam_log"] = np.log(data["csmam"] + 1)

In [16]:
data.columns

Index(['txkey', 'locdt', 'loctm', 'chid', 'cano', 'contp', 'etymd', 'mchno',
       'acqic', 'mcc', 'conam', 'ecfg', 'insfg', 'iterm', 'bnsfg', 'flam1',
       'stocn', 'scity', 'stscd', 'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'csmam',
       'flg_3dsmk', 'label', 'lochr', 'combined_datetime', 'contp_0',
       'contp_1', 'contp_2', 'contp_3', 'contp_4', 'contp_5', 'contp_6',
       'etymd_0.0', 'etymd_1.0', 'etymd_2.0', 'etymd_3.0', 'etymd_4.0',
       'etymd_5.0', 'etymd_7.0', 'etymd_8.0', 'etymd_9.0', 'etymd_10.0',
       'mcc_cat', 'mcc_cat_mcc1', 'mcc_cat_mcc2', 'mcc_cat_mcc3',
       'mcc_cat_mcc4', 'mcc_cat_mcc5', 'mcc_cat_unknown', 'conam_log',
       'flam1_log', 'stscd_0.0', 'stscd_1.0', 'stscd_2.0', 'stscd_3.0',
       'stscd_4.0', 'stscd_nan', 'hcefg_0.0', 'hcefg_1.0', 'hcefg_2.0',
       'hcefg_3.0', 'hcefg_4.0', 'hcefg_5.0', 'hcefg_6.0', 'hcefg_7.0',
       'hcefg_8.0', 'hcefg_9.0', 'hcefg_10.0', 'hcefg_nan', 'csmam_log'],
      dtype='object')

In [22]:
drop_col = ['txkey', 'locdt', 'loctm', 'chid', 'cano', 'contp', 'etymd', 'mchno',
       'acqic', 'mcc', 'conam','flam1',
       'stocn', 'scity', 'stscd','csmcu', 'csmam']

In [23]:
data1 = data.drop(drop_col, axis = 1).dropna()

In [24]:
X = data1.drop(['label'], axis = 1)
Y = data1['label']

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [27]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [28]:
rfc = RandomForestClassifier()

In [29]:
rfc = rfc.fit(xTrain, yTrain)

MemoryError: Unable to allocate 1.27 GiB for an array with shape (29, 5881309) and data type object